# The ancestral recombination graph (ARG)

In [1]:
import arg_dashboard
arg_dashboard.app.run_server(mode='inline', height=900, debug=True)

# Exercises

- For now just see if you can find your way around the dashboard...

In [ ]:
from arg_dashboard import app
import arg_dashboard

import arg_dashboard
from importlib import reload
reload(arg_dashboard)
arg_dashboard.app.run_server(mode='inline', height=900, debug=True)